# SQL Tutorial for The Movie Database (TMDb) Exam


## 1. Introduction

- Welcome
- Announcements
- Brief overview of SQL and its uses.
- Explanation of the TMDb database and its relevance.

### Overview of SQL and its uses

SQL, or Structured Query Language, is the standard language for dealing with relational databases.<br> It is used to insert, search, update, delete database records.<br> Essentially, it's how we communicate with a database to manage the data it holds.
<br>To begin working on relational database using our Jupyter notebook, remember we need to primarily do two things:
1. Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.


In [ ]:
%load_ext sql

2. Establish a connection to the local database using the '%sql' magic command.

In [ ]:
%sql sqlite:///TMDB-a-4006.db

'Connected: @TMDB-a-4006.db'

### Explanation of the TMDb database and its relevance:

The TMDb database is a comprehensive source for movie and TV show data, widely used for educational, research, and entertainment industry purposes.<br> Understanding how to query such databases can provide valuable insights and is an essential skill for data analysts, marketers, and developers alike.

Always remember that you can always do a detailed analysis on the most recent version of the TMDB database for your own portfolio.

The post on "How to Download and Explore Movie" Data by Steve Hedden, gives a detailed approach to downloading the TMDB movie database. click <a href="https://towardsdatascience.com/how-to-download-and-explore-movie-data-1948a887c530">here</a> to visit the post.

## 2. SQL Queries and Syntax

### Writing basic SELECT statements:

The SELECT statement is used to select data from a database. The data returned is stored in a result table, sometimes called the result set.

Example:<br>

`SELECT column1, column2
FROM table_name;`

That is the basic syntax of a query, but how do we know what tables to query, this will differ depending on the kind of db file and the dialect, since we are using sqlite, we can use the query below:

In [ ]:
%%sql
SELECT
    *
FROM
    sqlite_master
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


type,name,tbl_name,rootpage,sql
table,actors,actors,2,"CREATE TABLE `actors` ( `actor_id` integer NOT NULL, `actor_name` varchar(100) DEFAULT NULL, `gender` integer DEFAULT NULL, PRIMARY KEY (`actor_id`))"


### Importance of the WHERE clause:


The WHERE clause is used to filter records.<br>
It is used to extract only those records that fulfill a specified condition.

Example:  Let look at all types in the query above that are tables

In [ ]:
%%sql
SELECT
    *
FROM
    sqlite_master
WHERE
    type = 'table'
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


type,name,tbl_name,rootpage,sql
table,actors,actors,2,"CREATE TABLE `actors` ( `actor_id` integer NOT NULL, `actor_name` varchar(100) DEFAULT NULL, `gender` integer DEFAULT NULL, PRIMARY KEY (`actor_id`))"


In [ ]:
%%sql
SELECT
    name,
    type
FROM
    sqlite_master
WHERE
    type = 'table'
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


name,type
actors,table


### Between WHERE and HAVING

on the surface the WHERE and HAVING clause seem to do the same job but the major difference lies in the manner in which an SQL query is executed. When we execute a query in the background, SQL looks for all clauses present in the query and executes them in the following order

1. FROM
2. WHERE
3. GROUP BY
4. HAVING
5. SELECT
6. ORDER BY

The WHERE clause is processed/evaluated just after the required table, thus all rows are checked at this point. So we can see that the WHERE clause operates on all individual records (rows) in the data. However, the `HAVING` clause is evaluated after the `GROUP BY` clause. This means that the `HAVING` clause will evaluate on individual groups, not each row. This is the primary difference between both.

### Using ORDER BY and LIMIT:

The ORDER BY clause is used to sort the result set in either ascending or descending order. The LIMIT clause is used to specify the number of records to return.

Example:

In [ ]:
%%sql
SELECT
    name
FROM
    sqlite_master
WHERE
    type = 'table'
ORDER BY
    name
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


name
actors


#### Extra:
>Have you ever tried to check all columns in a table while working in SQLite?
>
>if you have reserached then  you should have come across the sqlite special function - "pragma_table_info"
>
>Since this is available, we can join the sqlite_master table with a pragma_table_info function to get all table names and columns in the database  
>
>Remember to filter for table.

In [ ]:
%%sql

SELECT
    m.name AS table_name,
    p.name AS column_name
FROM
    sqlite_master AS m
JOIN
    pragma_table_info(m.name) AS p
WHERE
    m.type = 'table'
ORDER BY
    m.name,
    p.cid
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


table_name,column_name
actors,actor_id


## 3. Understanding ERDs and Database Schema

- How to read an ERD.
- Understanding relationships between tables.
- Using primary and foreign keys for joining tables.

### Reading an ERD:

An Entity Relationship Diagram (ERD) is a visual representation of entities and their relationships to each other within a database.

![ALT TEXT](images/TMDB_ER_diagram.png)

[ALT TEXT](images/TMDB_ER_diagram.png)

### Understanding table relationships:
The relationships between tables in a database are crucial for understanding how data is interconnected. Key concepts include primary keys (unique identifiers for table records) and foreign keys (identifiers that link different tables together).

In [ ]:
%%sql

SELECT
    name,
    sql
FROM
    sqlite_master
WHERE
    type = 'table'
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


name,sql
actors,"CREATE TABLE `actors` ( `actor_id` integer NOT NULL, `actor_name` varchar(100) DEFAULT NULL, `gender` integer DEFAULT NULL, PRIMARY KEY (`actor_id`))"


## 4. Joining Tables

Types of joins:
SQL JOIN clause is used to combine rows from two or more tables, based on a related column between them.<br>
Common JOINS are

1. INNER
2. LEFT (OUTER) JOIN
3. RIGHT (OUTER) JOIN
4. FULL (OUTER) JOIN

#### INNER JOIN:

Use Case:
When you need to retrieve records that have matching values in both tables, an INNER JOIN is the most appropriate.<br> It is the most common type of join because it returns rows only when there is at least one match in both tables.

##### Example Question:
List the titles and release dates of all movies along with the names of the production companies that produced them.

###### Explanation:
In this scenario, we are interested in movies that are associated with production companies. The INNER JOIN ensures that we only get the movies that have a linked record in the productioncompanymap table (indicating that a production company is associated with the movie). This join filters out movies without production companies and production companies without movies, thus providing a clean list of movies with their respective producers.

In [ ]:
%%sql
-- only movies with titles and production companies because we used Inner to Join both tables
SELECT m.title, m.release_date, pc.production_company_name
FROM movies m
INNER JOIN
    productioncompanymap pcm
    ON m.movie_id = pcm.movie_id
INNER JOIN
    productioncompanies pc
    ON pcm.production_company_id = pc.production_company_id
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


title,release_date,production_company_name
Star Wars,1977-05-25 00:00:00.000000,Lucasfilm


#### LEFT JOIN (or LEFT OUTER JOIN)

Use Case:
A LEFT JOIN is used when you want all records from the left table (table before the JOIN keyword), and the matched records from the right table. The result is NULL from the right side if there is no match.

##### Example Question:
Provide a list of all movies, including their titles, release dates, and the names of their production companies, regardless of whether they have a production company associated with them or not.

###### Explanation:
The use of a LEFT JOIN here ensures that all movies are listed, even if they don't have an associated production company. This is useful when the completeness of the left-side table (in this case, the movies table) is essential, and you want to include all its records in the results.

In [ ]:
%%sql
-- This keeps all records from the table on the LEFT and adds records from table on the right where there is a PK - FK match.
-- If there are records where the PK - FK do not match, NULL are returned.
SELECT
    m.title,
    m.release_date,
    pc.production_company_name
FROM
    movies m
LEFT JOIN
    productioncompanymap pcm
    ON m.movie_id = pcm.movie_id
LEFT JOIN
    productioncompanies pc
    ON pcm.production_company_id = pc.production_company_id
limit 1;

 * sqlite:///TMDB-a-4006.db
Done.


title,release_date,production_company_name
Four Rooms,1995-12-09 00:00:00.000000,Miramax Films


#### RIGHT JOIN (or RIGHT OUTER JOIN)

Use Case:
A RIGHT JOIN is essentially the opposite of a LEFT JOIN. It returns all records from the right table, and the matched records from the left table. The result is NULL from the left side when there is no match. In systems that don't support RIGHT JOIN, like SQLite, the same effect can be achieved by reversing the order of tables and using a LEFT JOIN.

##### Example Question:
Show a list of all production companies and the movies they have produced, including the production companies that have not produced any movies.

###### Explanation:
In this instance, we are focused on the production companies, wanting a comprehensive list that includes those that may not have produced any movies. By using a RIGHT JOIN (or reversed LEFT JOIN in SQLite), we can list all production companies and match them with movies they've produced, ensuring even those without any movie production are represented.

In [ ]:
%%sql

SELECT
    pc.production_company_name,
    m.title,
    m.release_date
FROM productioncompanies pc
LEFT JOIN
    productioncompanymap pcm
    ON pc.production_company_id = pcm.production_company_id
LEFT JOIN
    movies m
    ON pcm.movie_id = m.movie_id
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


production_company_name,title,release_date
Lucasfilm,Star Wars,1977-05-25 00:00:00.000000


#### RIGHT JOIN (or RIGHT OUTER JOIN)

Use Case:
A RIGHT JOIN is essentially the opposite of a LEFT JOIN. It returns all records from the right table, and the matched records from the left table. The result is NULL from the left side when there is no match. In systems that don't support RIGHT JOIN, like SQLite, the same effect can be achieved by reversing the order of tables and using a LEFT JOIN.

##### Example Question:
Show a list of all production companies and the movies they have produced, including the production companies that have not produced any movies.

###### Explanation:
In this instance, we are focused on the production companies, wanting a comprehensive list that includes those that may not have produced any movies. By using a RIGHT JOIN (or reversed LEFT JOIN in SQLite), we can list all production companies and match them with movies they've produced, ensuring even those without any movie production are represented.

In [ ]:
%%sql

SELECT
    pc.production_company_name,
    m.title,
    m.release_date
FROM
    productioncompanies pc
LEFT JOIN
    productioncompanymap pcm
    ON pc.production_company_id = pcm.production_company_id
LEFT JOIN
    movies m
    ON pcm.movie_id = m.movie_id
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


production_company_name,title,release_date
Lucasfilm,Star Wars,1977-05-25 00:00:00.000000


#### FULL OUTER JOIN

Use Case:
FULL OUTER JOIN is used when you want to return all records when there is a match in either the left or right table. It combines the effects of both LEFT and RIGHT joins, including all records from both tables and filling in NULLs for unmatched rows from either side.

##### Example Question:
Find a list of all movies and all production companies, including those that do not have a match in the other table.

###### Explanation:
This scenario calls for a comprehensive list of both movies and production companies, regardless of whether they are associated with one another. A FULL OUTER JOIN will include all movies that might not be produced by any company in the list and all production companies, even those that haven't produced a listed movie. This type of join provides the most complete dataset from both tables.

In [ ]:
%%sql

SELECT
    m.title,
    m.release_date,
    pc.production_company_name
FROM
    movies m
LEFT JOIN
    productioncompanymap pcm
    ON m.movie_id = pcm.movie_id
LEFT JOIN
    productioncompanies pc
    ON pcm.production_company_id = pc.production_company_id

UNION

SELECT
    m.title,
    m.release_date,
    pc.production_company_name
FROM
    productioncompanies pc
LEFT JOIN
    productioncompanymap pcm
    ON pc.production_company_id = pcm.production_company_id
LEFT JOIN
    movies m
    ON pcm.movie_id = m.movie_id
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


title,release_date,production_company_name
#Horror,2015-11-20 00:00:00.000000,AST Studios


#### Summary:

In summary, the choice of JOIN type depends on the relationship between the tables and the completeness of the data required from each table in the query results. INNER JOIN is used for strict matches, LEFT JOIN is used when all records from the primary table are needed, RIGHT JOIN is for all records from the secondary table, and FULL OUTER JOIN is when all records from both tables are needed.

## 5. Aggregation and Grouping Data

- Using functions like COUNT(), AVG(), SUM(), MAX(), MIN().
- The concept of grouping data with GROUP BY.
- Having clause for filtering aggregated data.

### Aggregation:

Aggregation in SQL is performed using aggregate functions, which allow you to perform a calculation on a set of values and return a single value. Aggregate functions are
1. COUNT()
2. AVG()
3. SUM()
4. MAX()
5. MIN()

##### Example Question:
What is the total number of records, total number of movie titles, average duration of movies, longest and shortest movie duration, and how long it will take to watch all movies in our database?

In [ ]:
%%sql

SELECT
    COUNT(*) AS total_records,
    COUNT(DISTINCT title) AS total_unique_movie_titles,
    AVG(runtime) AS average_movie_duration,
    MAX(runtime) AS longest_movie_duration,
    MIN(runtime) AS shortest_movie_duration,
    SUM(runtime) AS total_time_required_to_watch_all_movies
FROM movies;


 * sqlite:///TMDB.db
Done.


total_records,total_unique_movie_titles,average_movie_duration,longest_movie_duration,shortest_movie_duration,total_time_required_to_watch_all_movies
4803,4800,106.87585919600083,338.0,0.0,513111.0


### Grouping Data

#### COUNT()

##### Question: How many movies has each actor appeared in?

###### Example Explanation: To determine the number of movies each actor has appeared in, we need to count the instances of movie appearances. Since an actor can appear in multiple movies, and a movie can have multiple actors, there's a many-to-many relationship, which is resolved by the casts table. We perform an INNER JOIN between actors and casts, and then casts with movies to relate the actors to their movies. We then use COUNT() to tally the number of movies associated with each actor, and GROUP BY to ensure the count is calculated for each actor individually.



In [ ]:
%%sql

SELECT
    a.actor_name,
    COUNT(m.movie_id) AS number_of_movies
FROM
    actors a
JOIN
    casts c
    ON a.actor_id = c.actor_id
JOIN
    movies m
    ON c.movie_id = m.movie_id
GROUP BY
    a.actor_name
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


actor_name,number_of_movies
Jorge de los Reyes,1


#### AVG()

##### Question: What is the average runtime of movies in each genre?

###### Example Explanation: When calculating the average runtime of movies by genre, we're looking for the mean value of the runtime across a category of movies. The genres and movies tables are connected via the genremap table, representing a many-to-many relationship. We join these tables to link each movie to its genre(s) and then use AVG() to find the average runtime for the movies within each genre. The GROUP BY clause is necessary to group the results by genre, as we want the average for each genre, not for the entire dataset.

In [ ]:
%%sql

SELECT
    g.genre_name,
    AVG(m.runtime) AS average_runtime
FROM
    genres g
JOIN
    genremap gm
    ON g.genre_id = gm.genre_id
JOIN
    movies m
    ON gm.movie_id = m.movie_id
GROUP BY
    g.genre_name
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


genre_name,average_runtime
Action,110.54419410745234


#### SUM()

##### Question: What is the total budget for movies produced by each production company?

###### Example Explanation: To compute the total budget spent by each production company on their movies, we need to sum the budget of all movies produced by each company. The relationship between productioncompanies and movies is many-to-many, mediated by the productioncompanymap table. We join these tables to associate each movie with its production company and use SUM() to calculate the total budget per production company. We group the results by production company name to get a separate sum for each one.



In [ ]:
%%sql

SELECT
    pc.production_company_name,
    SUM(m.budget) AS total_budget
FROM
    productioncompanies pc
JOIN
    productioncompanymap pcm
    ON pc.production_company_id = pcm.production_company_id
JOIN
    movies m ON pcm.movie_id = m.movie_id
GROUP BY
    pc.production_company_name
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


production_company_name,total_budget
"""""""DIA"""" Productions GmbH & Co. KG""",36000000


#### Having Clause

##### Question: Which actors have appeared in more than 20 movies?

###### Example Explanation: The HAVING clause is used to filter grouped records produced by GROUP BY. In this case, we want to identify actors who have appeared in a significant number of movies—more than 10. We join actors to movies via casts to establish the relationship between actors and the movies they've appeared in. After counting the movies per actor using COUNT() and grouping the results with GROUP BY, we apply the HAVING clause to filter our grouped results, keeping only those groups where the count exceeds 10. This allows us to exclude actors with fewer movie appearances from the result set.


In [ ]:
%%sql

SELECT
    a.actor_name,
    COUNT(m.movie_id) AS number_of_movies
FROM
    actors a
JOIN
    casts c ON a.actor_id = c.actor_id
JOIN
    movies m ON c.movie_id = m.movie_id
GROUP BY
    a.actor_name
HAVING
    COUNT(m.movie_id) > 20
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


actor_name,number_of_movies
Aaron Eckhart,21


## 6. Advanced Topics

- Subqueries and nested SELECT statements.
- Case statements and conditional logic in SQL.
- Functions for string manipulation and date handling.

### Subqueries and Nested SELECT Statements

#### Question: What are the titles of movies that have a higher popularity than the average popularity of movies released in the year 2010?

##### Explanation: A subquery is utilized here to first calculate the average popularity of all movies released in 2010. The subquery operates within the WHERE clause of the main query. The main query then compares each movie's popularity against this average, selecting only those movies whose popularity is greater. This type of subquery is also known as a scalar subquery because it returns a single value that the main query can use in a comparison.

In [ ]:
%%sql

SELECT title
FROM movies
WHERE popularity > (
    SELECT AVG(popularity)
    FROM movies
    WHERE release_date BETWEEN '2010-01-01' AND '2010-12-31'
)
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


title
Four Rooms


### Case Statements and Conditional Logic in SQL

#### Example Question:
For every movie, display the title, the total number of cast members, and a status column that indicates 'Well Cast' if there are more than 20 cast members, 'Moderately Cast' if there are between 11 and 20 cast members, and 'Poorly Cast' if there are less than 11 cast members.

##### Explanation:
This question involves counting the number of cast members for each movie and then categorizing each movie based on this count. The CASE statement is used to create a conditional logic that assigns a status based on the number of cast members associated with each movie.

In [ ]:
%%sql

SELECT
    m.title,
    COUNT(c.actor_id) AS total_cast,
    CASE
        WHEN COUNT(c.actor_id) > 20 THEN 'Well Cast'
        WHEN COUNT(c.actor_id) BETWEEN 11 AND 20 THEN 'Moderately Cast'
        ELSE 'Poorly Cast'
    END AS casting_status
FROM movies m
LEFT JOIN casts c ON m.movie_id = c.movie_id
GROUP BY m.title
LIMIT 1;

 * sqlite:///TMDB-a-4006.db
Done.


title,total_cast,casting_status
#Horror,9,Poorly Cast


### Functions for String Manipulation and Date Handling

##### Example Question: Find the number of movies released each month in the year 2015, along with the month name. Use string manipulation and date handling functions.

###### Explanation: This question combines date handling and string manipulation functions. The STRFTIME function is used to extract the month and year from a DATETIME column. The grouping is done by the month to count the number of movies released in each one. Additionally, a series of CASE statements translate the numeric month into its corresponding name. This question exemplifies how SQL can manipulate and format date-time data to produce a more readable and informative result set.

In [ ]:
%%sql

SELECT
    STRFTIME('%m', release_date) AS month_number,
    STRFTIME('%Y', release_date) AS year,
    CASE
        WHEN STRFTIME('%m', release_date) = '01' THEN 'January'
        WHEN STRFTIME('%m', release_date) = '02' THEN 'February'
        WHEN STRFTIME('%m', release_date) = '03' THEN 'March'
        WHEN STRFTIME('%m', release_date) = '04' THEN 'April'
        WHEN STRFTIME('%m', release_date) = '05' THEN 'May'
        WHEN STRFTIME('%m', release_date) = '06' THEN 'June'
        WHEN STRFTIME('%m', release_date) = '07' THEN 'July'
        WHEN STRFTIME('%m', release_date) = '08' THEN 'August'
        WHEN STRFTIME('%m', release_date) = '09' THEN 'September'
        WHEN STRFTIME('%m', release_date) = '10' THEN 'October'
        WHEN STRFTIME('%m', release_date) = '11' THEN 'November'
        WHEN STRFTIME('%m', release_date) = '12' THEN 'December'
    END AS month_name,
    COUNT(*) AS movies_released
FROM
    movies
WHERE
    STRFTIME('%Y', release_date) = '2015'
GROUP BY
    month_number, month_name
LIMIT 1;


 * sqlite:///TMDB-a-4006.db
Done.


month_number,year,month_name,movies_released
01,2015,January,13


##### Example Question: Find all movies where the title starts with 'The' followed by any characters, and ends with the word 'Club', including any possible spaces or punctuation before 'Club'.



###### Explanation:
The LIKE operator in SQL is used to search for a specified pattern in a column. In this case, we want to find movie titles that fit a particular naming pattern. We will use the % wildcard, which represents zero or more characters, and _ which represents a single character. To account for possible spaces or punctuation, we'll use a combination of these wildcards.

In [ ]:
%%sql

SELECT
    title
FROM
    movies
WHERE
    title LIKE 'The%Club'
    OR title LIKE 'The% Club'
    OR title LIKE 'The%_Club';

 * sqlite:///TMDB.db
Done.


title
The Cotton Club
The First Wives Club
The Emperor's Club
The Players Club


## 7. Tips for tackling common SQL problems.

1. **Understand the Database Schema**: Before writing queries, always review the database schema. Knowing how tables are structured and how they relate to each other is crucial for writing effective SQL.

2. **Start with SELECT**: When building a query, start with a simple `SELECT` statement to ensure you're pulling the correct data. Gradually add complexity as needed.

3. **Use Table Aliases**: When working with multiple tables, especially with joins, use aliases to make your query easier to read and to avoid column ambiguity errors.

4. **Filter with WHERE Clauses**: Use WHERE clauses to filter data early in your query. This improves performance by reducing the amount of data that SQL operations need to process in later stages.

5. **Break Down Complex Queries**: If you're dealing with a complex query, break it down into smaller parts and tackle each part individually. This can help you isolate issues and understand the data better.

6. **Make Use of Subqueries**: Sometimes, creating a subquery for a portion of your data is necessary before you can perform the main query. This can simplify complex operations.

7. **Leverage Aggregation Functions**: Understand and use aggregation functions (`COUNT`, `SUM`, `AVG`, `MIN`, `MAX`) effectively, along with `GROUP BY` to summarize data.

8. **Test with LIMIT**: When working with large tables, use `LIMIT` to test your queries with a small data set first, which can save time and resources.

9. **Use CASE for Conditional Logic**: For more nuanced data retrieval, use the `CASE` statement to handle conditional logic and derive new columns based on conditions.

10. **Practice String Manipulation**: Familiarize yourself with the various SQL string functions (`LIKE`, `CONCAT`, `SUBSTRING`, etc.) to handle and search text data efficiently.

11. **Handle Dates Carefully**: Dates can be tricky in SQL. Make sure to use the correct date format and understand how your SQL database handles dates. Use date functions (`NOW()`, `DATE()`, `DATEDIFF()`, etc.) for manipulating date-time values.

12. **Avoid SELECT** * : Instead of using `SELECT *` in your final query, specify the exact columns you need. This can improve performance and prevent unintended data exposure.

13. **Optimize JOINs**: Be mindful of the join types (`INNER`, `LEFT`, `RIGHT`, `FULL`) and only pull the necessary data you need. Unnecessary joins can greatly decrease query performance.

14. **Validate Data Types**: Ensure that the data types you are comparing or calculating are compatible; otherwise, you may run into errors or unexpected results.

15. **Use Comments**: Comment your SQL code to clarify complex logic. This is especially helpful when you or someone else revisits the query later.

16. **Error Handling**: Learn how to read and understand error messages. They often contain the key to resolving the problem.

17. **Consistent Formatting**: Write your SQL with consistent capitalization and indentation. This makes it easier to read and understand, especially for more complex queries.

18. **Research and Resources**: Make use of the vast amount of online resources available, such as SQL forums, documentation, and interactive SQL platforms for learning and troubleshooting.

19. **Performance Tuning**: Learn to use EXPLAIN plans to understand how your queries are executed and optimize them for better performance.

20. **Regular Practice**: Like any language, proficiency in SQL comes with regular practice. Work on diverse problems to strengthen your understanding and adaptability.

By keeping these tips in mind, you can approach SQL problems methodically and efficiently, increasing their chances of successfully writing queries that yield the correct results.